In [ ]:
import pandas as pd
from gensim import utils
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from gensim.parsing.preprocessing import preprocess_string, remove_stopwords
import random

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
movie_lines_file = 'movie_lines.txt'

In [ ]:
with open(movie_lines_file) as f:
    movie_lines = [line.strip().split('+++$+++') for line in f.readlines()];

lines_df = pd.DataFrame([{'LineNumber': d[0].strip(), 
                                'Person': d[3].strip(),
                                'Line': d[4].strip(),
                                 'Movie' : d[2].strip()} 
                              for d in movie_lines])
lines_df = lines_df.set_index('LineNumber')

In [ ]:
lines_df.head(10)

In [ ]:
len(lines_df)

In [ ]:
lines_df.nunique()

In [ ]:
lines_df_small = lines_df.head(50000)

In [ ]:
class DocumentDataset(object):
    
    def __init__(self, data:pd.DataFrame, column):
        document = data[column].apply(self.preprocess)
        self.documents = [ TaggedDocument( text, [index]) 
                          for index, text in document.iteritems() ]
      
    def preprocess(self, document):
        return preprocess_string(remove_stopwords(document))
        
    def __iter__(self):
        for document in self.documents:
            yield documents
            
    def tagged_documents(self, shuffle=False):
        if shuffle:
            random.shuffle(self.documents)
        return self.documents

In [ ]:
documents_dataset = DocumentDataset(lines_df_small, 'Line')

In [ ]:
documents_dataset

In [ ]:
docVecModel = Doc2Vec(min_count=1, window=5, vector_size=100, sample=1e-4, negative=5, workers=8)
docVecModel.build_vocab(documents_dataset.tagged_documents())

In [ ]:
docVecModel.train(documents_dataset.tagged_documents(shuffle=True),
            total_examples = docVecModel.corpus_count,
           epochs=10)

In [ ]:
docVecModel['L1045']

In [ ]:
docVecModel['L872']

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def show_image(vector, line):
    fig, ax = plt.subplots(1,1, figsize=(10, 2))
    ax.tick_params(axis='both', 
                   which='both',
                   left=False, 
                   bottom=False,
                   top=False,
                   labelleft=False,
                   labelbottom=False)
    ax.grid(False)
    print(line)
    ax.bar(range(len(vector)), vector, 0.5)
   
  
def show_movie_line(line_number):
    line = lines_df_small.ix['L872'].Line
    doc_vector = docVecModel[line_number]
    show_image(doc_vector, line)

In [ ]:
show_movie_line('L872')